In [ ]:
!pip install tushare
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.8/142.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 56.0 MB/s eta 0:00:00


In [ ]:
!pip install nvidia-smi
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
import psutil

ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 359.2 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import pandas as pd
import numpy as np
import tushare as ts
import os
from tqdm import tqdm
import glob
ts.set_token('ce1d8f98f48f0c92fa70a66528612510f350b82ea565bc1abd45b5ac')
debug = False
from datetime import datetime
pro = ts.pro_api()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# df = pd.read_csv("/content/drive/MyDrive/solution/input/train.csv")
#/content/drive/MyDrive/alpha_python_202505/alpha_python_202505/part01-download-data.ipynb


Mounted at /content/drive


## 股票基础信息

In [ ]:
stocklist = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [ ]:
stocklist.head(100)

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129
2,000004.SZ,000004,*ST国华,深圳,软件服务,19910114
3,000006.SZ,000006,深振业A,深圳,区域地产,19920427
4,000007.SZ,000007,全新好,深圳,其他商业,19920413
...,...,...,...,...,...,...
95,000509.SZ,000509,华塑控股,四川,元器件,19930507
96,000510.SZ,000510,新金路,四川,化工原料,19930507
97,000513.SZ,000513,丽珠集团,广东,化学制药,19931028
98,000514.SZ,000514,渝开发,重庆,区域地产,19930712


## 交易日历

In [ ]:
trade_cal = pro.trade_cal(exchange='', start_date='20190101', end_date='20250205')
trade_cal = trade_cal[trade_cal.is_open==1]
trade_cal.head()

,exchange,cal_date,is_open,pretrade_date
0,SSE,20250205,1,20250127
9,SSE,20250127,1,20250124
12,SSE,20250124,1,20250123
13,SSE,20250123,1,20250122
14,SSE,20250122,1,20250121


In [ ]:
trade_cal_1 = df = pro.query('trade_cal', exchange='', start_date='20190101', end_date='20250205', fields='exchange,cal_date,is_open,pretrade_date', is_open='1')
trade_cal_1.head(10)

,exchange,cal_date,is_open,pretrade_date
0,SSE,20250205,1,20250127
1,SSE,20250127,1,20250124
2,SSE,20250124,1,20250123
3,SSE,20250123,1,20250122
4,SSE,20250122,1,20250121
5,SSE,20250121,1,20250120
6,SSE,20250120,1,20250117
7,SSE,20250117,1,20250116
8,SSE,20250116,1,20250115
9,SSE,20250115,1,20250114


## 行情数据

In [ ]:
price_PD = []
for trade_date in tqdm(trade_cal.cal_date):
    df = pro.daily(trade_date=trade_date)
    price_PD.append(df)

price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv(f'data/content/drive/MyDrive/alpha_python_202505/alpha_python_202505/price_PD.csv', index=False)
# /content/drive/MyDrive/alpha_python_202505/alpha_python_202505/part01-download-data.ipynb



100%|██████████| 1475/1475 [39:15<00:00,  1.60s/it]


OSError: Cannot save file into a non-existent directory: 'data/content/drive/MyDrive/alpha_python_202505/alpha_python_202505'

## 重要指标

In [ ]:
price_PD = []
for trade_date in tqdm(trade_cal.cal_date):
    df = pro.daily_basic(ts_code='', trade_date=trade_date, \
                     fields='ts_code,trade_date,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,\
                     ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv')
    price_PD.append(df)

price_PD = pd.concat(price_PD).reset_index(drop=True)

price_PD = price_PD.drop_duplicates().sort_values(['ts_code','trade_date']).reset_index(drop=True)
price_PD.to_csv(f'data/daily_basic.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████| 1247/1247 [18:50<00:00,  1.10it/s]


In [1]:
import torch, torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Put the model on the GPU
model = nn.Linear(1000, 500).to(device)

# 2. Put the data on the same GPU *before* each forward pass
batch = torch.randn(64, 1000, device=device)

output = model(batch)          # runs on GPU
loss   = output.sum()
loss.backward()                # gradients on GPU


In [2]:
output

tensor([[ 0.5579,  0.5058,  1.1033,  ..., -1.0139,  0.8899,  0.4621],
        [-0.3189, -0.4425, -0.4545,  ...,  0.7136, -0.3340,  0.8076],
        [ 1.2152, -0.4023,  0.5696,  ..., -1.1942, -0.3525,  0.8798],
        ...,
        [-0.2064, -0.0812, -1.5391,  ..., -0.1278, -0.0793, -0.6622],
        [-0.2700, -0.1773, -0.5025,  ..., -0.1510,  0.3948, -0.0257],
        [ 0.4878, -1.1454,  0.1152,  ...,  0.2844, -0.2244, -0.0849]],
       grad_fn=<AddmmBackward0>)